In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [1]:
import os
import pandas as pd
import json

# Retrieve data
___
Sourced from Golden Cheetah Open Data Project

In [18]:
if 'apple' in os.environ['BUILD']:
    root_dir = '../gc_opendata-samples'
else:
    root_dir = 'E:\gc_opendata'

In [66]:
class open_dataset(object):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.athlete_ids = self.get_athlete_ids()
    def get_athlete_ids(self):
        for a,b,c in os.walk(self.root_dir):
            athletes = b
            athletes.remove('INDEX') if 'INDEX' in athletes else 0
            break
        return athletes

    def get_athlete_summary(self, athlete_id):
        ath_summary_path = self._athlete_summary_path(athlete_id)
        with open(ath_summary_path, 'r') as f:
            summary_json = f.read()
        f.close()
        rides = json.loads(summary_json)['RIDES']
        # ride_dict = summary_json['RIDES']
        df = pd.json_normalize(rides)
        return df
    
    def get_athlete_activity_files(self, athlete_id):
        
    
    def _athlete_dir(self, athlete_id):
        athlete_dir = os.path.join(self.root_dir, athlete_id)
        return athlete_dir

    def _athlete_summary_path(self, athlete_id)
        summary_filename = "{" + athlete_id + "}.json"
        ath_summary_path = os.path.join(self.root_dir, athlete_id, summary_filename)
        return ath_summary_path

In [67]:
od = open_dataset(root_dir)

In [68]:
s = od.get_athlete_summary(od.athlete_ids[0])

,date,data,sport,METRICS.ride_count,METRICS.workout_time,METRICS.time_recording,METRICS.time_riding,METRICS.time_carrying,METRICS.elevation_gain_carrying,METRICS.total_distance,...,METRICS.power_dist_bins,METRICS.power_mmp,METRICS.power_mmp_secs,METRICS.90m_critical_power,METRICS.antiss_score,METRICS.time_in_zone_L5,METRICS.skiba_wprime_max,METRICS.skiba_wprime_exp,METRICS.skiba_wprime_watts,METRICS.wcptime_in_zone_L1
0,2009/02/24 17:38:43 UTC,TDS--C-AGL-----,Bike,1.00000,12350701.00000,478.00000,457.00000,22.00000,3.40000,1.76350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009/07/17 16:26:13 UTC,TDS--C-AGL-----,Bike,1.00000,1486.00000,965.00000,934.00000,27.00000,NaN,3.82200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009/07/18 15:40:19 UTC,TDS--C-AGL-----,Bike,1.00000,86293.00000,765.00000,710.00000,45.00000,3.12000,2.58150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009/07/19 16:19:41 UTC,TDS--C-AGL-----,Bike,1.00000,2280.00000,2057.00000,1924.00000,41.00000,NaN,8.85630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009/07/21 15:37:05 UTC,TDS--C-AGL-----,Bike,1.00000,6568.00000,6079.00000,5991.00000,116.00000,NaN,27.58200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,2020/03/26 17:46:37 UTC,T--PHC-A-------,Bike,1.00000,4885.00000,4316.00000,4126.00000,NaN,NaN,NaN,...,"[0, 10, 20, 70, 80, 90, 100, 110, 120, 130, 14...","[272, 270, 261, 260, 257, 256, 255, 255, 253, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",NaN,0.00021,"[5.00000, 4316.00000]",0.61665,"[0.12400, 15.00000]","[0.02873, 4316.00000]",15.00000
3011,2020/03/26 19:12:42 UTC,T--PHC-A-------,Bike,1.00000,6284.00000,6288.00000,6281.00000,NaN,NaN,NaN,...,"[0, 10, 20, 30, 40, 70, 80, 90, 100, 110, 120,...","[192, 186, 183, 184, 183, 182, 181, 180, 179, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",120.79407,NaN,NaN,NaN,NaN,NaN,NaN
3012,2020/03/27 18:23:03 UTC,T--PHC-A---E---,Bike,1.00000,3604.00000,3604.00000,3602.00000,NaN,NaN,NaN,...,"[0, 10, 40, 50, 60, 70, 80, 90, 100, 110, 120,...","[156, 149, 146, 143, 142, 142, 142, 142, 140, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3013,2020/03/28 15:51:08 UTC,T--PHC-A---E---,Bike,1.00000,5429.00000,5429.00000,5428.00000,NaN,NaN,NaN,...,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[227, 227, 225, 224, 223, 222, 222, 221, 221, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",132.27064,0.00002,NaN,NaN,NaN,NaN,NaN


In [ ]:
target_activity = activities[-1]
with open(f'{root_dir}\{target_athlete}\{target_activity}', 'r') as f:
    ath_overview = f.read()

In [ ]:
slip = json.loads(ath_overview)

In [ ]:
slip = pd.DataFrame(pd.json_normalize(slip['RIDES']))

In [ ]:
slip

In [ ]:
activ_sum = pd.read_csv("C:/Users/ryand/git/TriHealth/activities.csv",parse_dates=['date'])

In [ ]:
activ_sum[['id','file']].groupby('id').count().sort_values('file').tail(30)

# Sample Explore

In [ ]:
target_id = "7567ac31-d479-4b25-9538-541ae8085288"

In [ ]:
act_samp = activ_sum[activ_sum['id'] == target_id].copy()

In [ ]:
act_samp.columns.tolist()

In [ ]:
act_samp['athlete_weight'] = act_samp['20m_critical_power']/act_samp['20m_peak_wpk']
act_samp['athlete_weight'] = act_samp['athlete_weight'].ffill().bfill()

In [ ]:
act_samp.set_index('date',inplace=True)

In [ ]:
act_samp[act_samp['sport'] == 'Bike']['20m_critical_power'].rolling(window=90, min_periods=1).max().plot();

In [ ]:
act_samp.plot(kind='scatter',x='workout_time',y='average_hr')

In [ ]:
def calc_vo2(row, max_hr, resting_hr):
    if row['sport'] == 'Bike':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (((row['average_power']/75)*1000)/row['athlete_weight']) / percent_vo2
        # vo2_estimated = row['VO2max_Detected'] ## overide with Garmin VO2 Estimation
    elif row['sport'] == 'Run':
        percent_vo2 = (row['average_hr'] - resting_hr)/(max_hr - resting_hr)
        vo2_estimated = (210/row['average_speed']) / percent_vo2
    else:
        vo2_estimated =  0
    return vo2_estimated

In [ ]:
act_samp['VO2'] = act_samp.apply(lambda row: calc_vo2(row, 179, 45), axis=1)

In [ ]:
act_samp

# Modeling

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
         
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)
    
net = NN()
print(net)